In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
# Creación de objeto pandas dataframe
patients_df=pd.read_csv('https://github.com/stedy/Machine-Learning-with-R-datasets/blob/master/insurance.csv?raw=true')

In [ ]:
# Creación variables binarias
patients_df.replace({'sex':{'male':0,'female':1}}, inplace=True)
patients_df.replace({'smoker':{'yes':1,'no':0}}, inplace=True)

In [ ]:
# La función get dummies convierte un DataFrame de columnas categoricas a uno con variables dummy variables
region_dummies_df=pd.get_dummies(patients_df[['region']])
region_dummies_df

In [ ]:
# Hacemos join entre los 2 dataframes para reconstruir el dataset
patients_df = patients_df.join(region_dummies_df)

In [ ]:
patients_df

In [ ]:
# Uso 70% para entrenamiento (random split)
train_df= patients_df.sample(frac=0.7,random_state=200)
rest_df = patients_df.drop(train_df.index)
# Uso 15% para validacion y 15% para test
val_df=rest_df.sample(frac=0.5,random_state=200)
test_df=rest_df.drop(val_df.index)

In [ ]:

def fit_model(X, Y, lr=0.00001, epochs=100, patience=10): 
    """Implementación de la función de entrenamiento por descendo de gracdiente

    Args:
        X (np array): vector de características nxm
        Y (np array): vector de variable objetivo
    """

    #Generacion de los thetas aleatorios
    n, m = X.shape
    theta = np.random.rand(m+1,1)
    #Se agrega la dimension
    X_c = np.hstack((np.ones((n,1)),X))
    loss_v = []
    best_loss = np.inf

    for epoch in range(epochs):
        Y_est = X_c.dot(theta)
        #Calcular la perdidad
        loss = np.sum(np.power(Y_est-Y,2))/(2.*n)
        loss_v.append(loss)
        #calculo gradientes
        gradientes = (-1/n)*(X_c.T.dot((Y - X_c.dot(theta))))
        #actualizar
        theta = theta - lr*gradientes

        # Esto se agrega para parar el entrenamiento en caso de que la perdida no disminuya por mas del valor del parametro patience
        if loss < best_loss:
            best_loss = loss
            best_theta = np.copy(theta)
            epochs_stall = 0
        else:
            epochs_stall+=1
        if epochs_stall>=patience:
            print('La funcion de perdida no ha disminuido, parando despues de {} epocas. el error es: {}'.format(epoch, loss))
            break
        print('Epoch: {} Loss: {:.4e}'.format(epoch, loss))

    print('El error fue: {:.4e}'.format(loss))
    return best_theta, loss_v
